In [1]:
#调用库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import random
import pylab
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import Canvas, Button
from PIL import Image, ImageDraw,ImageGrab
from PIL import ImageOps

In [2]:
#定义超参数
batch_sizes = 64 #每批处理数量
device = torch.device("cuda") #利用GPU计算
epochs = 20  #设置迭代次数为 10



In [3]:
#对图像处理
pipeline = transforms.Compose([
    transforms.ToTensor(), #将图片转型为tesnor格式
    transforms.Normalize((0.1307,),(0.3081,)) #降低模型过度拟合,即降低模型复杂度
    
    
])

In [4]:
from torch.utils.data import DataLoader

1.#下载mnist训练集进入文件
train_set= datasets.MNIST("mnist_data", train=True, download= True, transform=pipeline)

2.#下载mnist测试集进入文件
test_set = datasets.MNIST("mnist_data", train=False, download= True, transform=pipeline)

#加载数据
train_loader = DataLoader(train_set, batch_size= batch_sizes, shuffle=True)
test_loader = DataLoader(test_set, batch_size= batch_sizes, shuffle=True)

In [5]:
def get_train_data():
    train_set= datasets.MNIST("mnist_data", train=True, download= True, transform=pipeline)
    train_loader = DataLoader(train_set, batch_size= batch_sizes, shuffle=True)
    
    return train_set, train_loader
    

In [6]:
#构建神经网络try1模型
class neural_network_model_try(nn.Module):
    def __init__(self ):
        super(). __init__()
        self.cv1 = nn.Conv2d(1, 10 ,5) 
        self.cv2 = nn.Conv2d(10, 20 ,3 )
        self.fc1 = nn.Linear(20*10*10, 500 )
        self.fc2 = nn.Linear(500, 10)
    def forward (self, x):
        input_size = x.size(0)
        x = self.cv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)
        x = self.cv2(x)
        x = F.relu(x)
        x = x.view(input_size, -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        outputs = F.log_softmax(x, dim=1)
        
        return outputs
    
       
    
    

In [7]:
#定义优化器
model = neural_network_model_try().to(device)

optimizer = optim.Adam(model.parameters())

In [8]:
#训练模型
def train_model(model, device, train_loader, optimizer, epoch ):
    model.train()
    for batch_index, (data , target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        pred = output.max(1, keepdim=True)
        loss.backward()
        optimizer.step()
        if batch_index % 3500 == 0 :
            print("Train Epoch : {} \t Loss : {:.6f} ".format(epoch, loss.item()))

In [9]:
#测试模型
def test_model(model, device, test_loader ):
    model.eval()
    correct = 0.0
    test_loss = 0.0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss = F.cross_entropy(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /=len(test_loader.dataset) 
        print("Test -- Average loss : {: 4f} , Accuracy : {: 3f} \n ".format(
               test_loss, 100.0* correct / len(test_loader.dataset)))
   

In [10]:
#迭代测试
for epochs in range (1, epochs +1):
    train_model(model, device, train_loader, optimizer, epochs)
    test_model(model, device, test_loader)
   

Train Epoch : 1 	 Loss : 2.294174 
Test -- Average loss :  0.000000 , Accuracy :  98.330000 
 
Train Epoch : 2 	 Loss : 0.156089 
Test -- Average loss :  0.000006 , Accuracy :  98.960000 
 
Train Epoch : 3 	 Loss : 0.028014 
Test -- Average loss :  0.000000 , Accuracy :  98.920000 
 
Train Epoch : 4 	 Loss : 0.005039 
Test -- Average loss :  0.000000 , Accuracy :  98.800000 
 
Train Epoch : 5 	 Loss : 0.005430 
Test -- Average loss :  0.000000 , Accuracy :  98.840000 
 
Train Epoch : 6 	 Loss : 0.001146 
Test -- Average loss :  0.000000 , Accuracy :  98.880000 
 
Train Epoch : 7 	 Loss : 0.001109 
Test -- Average loss :  0.000000 , Accuracy :  99.040000 
 
Train Epoch : 8 	 Loss : 0.000155 
Test -- Average loss :  0.000000 , Accuracy :  98.910000 
 
Train Epoch : 9 	 Loss : 0.000067 
Test -- Average loss :  0.000000 , Accuracy :  99.000000 
 
Train Epoch : 10 	 Loss : 0.000559 
Test -- Average loss :  0.000001 , Accuracy :  98.980000 
 
Train Epoch : 11 	 Loss : 0.000350 
Test -- Avera

In [11]:
#可视化ui界面
def update_result_label(result_label, predicted_digit):
    result_label.config(text=f"Predicted Digit: {predicted_digit}")

class DigitRecognizerApp:
    def __init__(self, root, model):
        self.root = root
        self.model = model

        self.canvas = Canvas(self.root, width=280, height=280, background="white")
        self.canvas.pack()

        self.clear_button = Button(self.root, text="Clear", command=self.clear_canvas)
        self.clear_button.pack()

        self.recognize_button = Button(self.root, text="Recognize", command=self.recognize_digit)
        self.recognize_button.pack()

        self.result_label = tk.Label(self.root, text="Draw a digit and click 'Recognize'", font=("Helvetica", 16))
        self.result_label.pack()

        self.last_x = None
        self.last_y = None
        self.canvas.bind("<Button-1>", self.start_draw)
        self.canvas.bind("<B1-Motion>", self.draw)

    def clear_canvas(self):
        self.canvas.delete("all")
        self.result_label.config(text="Draw a digit and click 'Recognize'")

    def recognize_digit(self):
        x0 = self.canvas.winfo_rootx() + self.canvas.winfo_x()
        y0 = self.canvas.winfo_rooty() + self.canvas.winfo_y()
        x1 = x0 + self.canvas.winfo_width()
        y1 = y0 + self.canvas.winfo_height()

        image = ImageGrab.grab(bbox=(x0, y0, x1, y1))
        image = image.resize((28, 28), Image.ANTIALIAS)
        image = image.convert('L')
        image = ImageOps.invert(image)

        tensor_image = transforms.ToTensor()(image)
        tensor_image = tensor_image.unsqueeze(0).to(device)

        with torch.no_grad():
            output = self.model(tensor_image)
            predicted_digit = output.argmax().item()

        self.result_label.config(text=f"Predicted Digit: {predicted_digit}")

    def start_draw(self, event):
        self.last_x = event.x
        self.last_y = event.y

    def draw(self, event):
        x, y = event.x, event.y
        if self.last_x is not None and self.last_y is not None:
            self.canvas.create_line((self.last_x, self.last_y, x, y), fill="black", width=8)
            self.last_x = x
            self.last_y = y

if __name__ == "__main__":
    root = tk.Tk()
    app = DigitRecognizerApp(root, model)

    root.mainloop()

D:\Ai_Project_Tool\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
